In [1]:
import numpy as np
import scipy.sparse as sparse
import time
from scipy.sparse.linalg import spsolve
import logging

## Load Data

In [3]:
# reference
def load_tuples(filename):
    t0 = time.time()
    rating_tuple = []
    for i, line in enumerate(open(filename, 'r')):
        user, item, count = line.strip().split(',')
        user = int(float(user))
        item = int(float(item))
        count = float(count)
        rating_tuple.append([user,item,count])
        if i % 1000000 == 0:
            print('loaded %i counts...' % i)
    t1 = time.time()
    print('Finished loading tuples in %f seconds' % (t1 - t0))
    return rating_tuple
def load_matrix(rating_tuple, num_users, num_items):
    t0 = time.time()
    counts = sparse.dok_matrix((num_users, num_items), dtype=float)
    for i, r_tuple in enumerate(rating_tuple):
        user, item, count = r_tuple
        if user >= num_users:
            continue
        if item >= num_items:
            continue
        if count != 0:
            counts[user, item] = count
        if i % 1000000 == 0:
            print('loaded %i counts...' % i)
    counts = counts.tocsr()
    t1 = time.time()
    print('Finished loading matrix in %f seconds' % (t1 - t0))
    return counts

In [4]:
train_file = "./data/select_data/train_tuples.txt"
im_file = "./data/select_data/im_test_tuples.txt"
om_file = "./data/select_data/om_test_tuples.txt"

train_tuple = load_tuples(train_file)
im_tuple = load_tuples(im_file)
om_tuple = load_tuples(om_file)

num_users = int(np.max(np.array(train_tuple)[:,0])) + 1
num_items = int(np.max(np.array(train_tuple)[:,1])) + 1
print("Loading train matrix")
train_counts = load_matrix(train_tuple,num_users,num_items)

loaded 0 counts...
Finished loading tuples in 2.489589 seconds
loaded 0 counts...
Finished loading tuples in 0.392614 seconds
loaded 0 counts...
Finished loading tuples in 0.372670 seconds
Loading train matrix
loaded 0 counts...
Finished loading matrix in 9.169604 seconds


In [5]:
print(len(train_tuple))
train_counts.shape

567308


(142517, 111176)

## Model

In [6]:
class CF():
    def __init__(self,ratings,num_users, num_products, K=200, lambda_u=0.01, lambda_v=0.01, n_iter=1, alpha = 1, checkpoint = None,idx = None):
        self.K = K
        self.n_iter = n_iter
        self.lambda_u = lambda_u
        self.lambda_v = lambda_v
        self.alpha = alpha
        
        self.num_users = num_users
        self.num_products = num_products
        #self.ratings = self.load_matrix(ratings)
        self.ratings = ratings
        if(checkpoint):
            t0 = time.time()
            self.U = sparse.load_npz(checkpoint+"/matrix_u"+idx+".npz")
            self.V = sparse.load_npz(checkpoint+"/matrix_v"+idx+".npz")
            t1 = time.time()
            print("load UV matrix from last checkpoint in {} seconds".format(t1-t0))
        else:
            t0 = time.time()
            self.U = np.sqrt(1.0/lambda_u)*sparse.csr_matrix(sparse.random(self.num_users, K, density=0.9, data_rvs=np.random.randn))
            self.V = np.sqrt(1.0/lambda_v)*sparse.csr_matrix(sparse.random(self.num_products,K, density=0.9, data_rvs=np.random.randn))
#             self.U = sparse.csr_matrix(np.sqrt(1.0/lambda_u) * sparse.random(self.num_users, K, density=0.5, data_rvs=np.random.randn))
#             self.V = sparse.csr_matrix(np.sqrt(1.0/lambda_v) * sparse.random(self.num_products,K, density=0.5, data_rvs=np.random.randn))
            t1 = time.time()
            print("generate random UV in {} seconds".format(t1-t0))
            
    def fit(self):
        for i in range(self.n_iter):
            # Update user vector
            self.U = self.update("user")
            #  Update product vector
            self.V = self.update("product")
    def update(self,target_type):
        print('Begin update '+target_type)
        t0 = time.time()
        if target_type == "user":
            num = self.num_users
            Y = self.V
            lambda_ = self.lambda_u
        else:
            num = self.num_products
            Y = self.U
            lambda_ = self.lambda_v
        num_fixed = Y.shape[0]
        YTY = Y.T.dot(Y) 
        # accumulate YtCuY + regularization*I in A
        A = YTY + lambda_ * sparse.eye(self.K)
        # accumulate YtCuPu in b
        b = sparse.csr_matrix((self.K,1))
        # placeholder for solution
        X = sparse.lil_matrix((num, self.K))
        for i in range(num):
            #print(i)
            Ri = self.ratings[i].T if target_type =="user" else self.ratings[:,i]
            X[i] = self.solve_equation(A,b,Y,Ri,target_type)
            if(i%5000 == 0):
                print(time.strftime("%H:%M:%S", time.localtime()) + ": The {}th {} ".format(i, target_type))
        t1 = time.time()
        print('Finished update '+target_type+' in %f seconds' % (t1 - t0))
        return X.tocsr()
            
    def solve_equation(self,A,b,Y,Ri,target_type):
        # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
        # YtCuY + regularization * I = YtY + regularization * I + Yt(Cu-I)

        # accumulate YtCuY + regularization*I in A
        # accumulate YtCuPu in b
        #import ipdb;ipdb.set_trace()
        for j in Ri.nonzero()[0]:
            factor = Y[j]
            r = Ri[j].data[0]
            #confidence = 1+self.alpha*r
            if r > 0:     
                #b += confidence * factor.T
                assert (factor.T*factor).shape != (1,1)
                #A += (confidence - 1) * factor.T*factor 
                b += r * factor.T
                A += factor.T*factor 
        #print(b.shape)
        #print(A.shape)
        #print("----",b.size)
        x = spsolve(A, b)
        return x
    
    def predict_rate(self,user_id, product_id):
        if(user_id >= self.num_users):
            #print("new users")
            u_vector = sparse.csr_matrix(self.U[0].shape, dtype=float)
        else:
            u_vector = self.U[user_id]
        if(product_id >= self.num_products):
            #print("new products")
            v_vector = sparse.csr_matrix(self.V[0].shape, dtype=float)
        else:
            v_vector = self.V[product_id]
        return (u_vector.dot(v_vector.T)).data
    
    def MSE(self,test):
        t0 = time.time()
        error = []
        for user,product,target_rate in test:
            r = self.predict_rate(user,product)
            if(len(r)):
                error.append((r-target_rate)**2)
        t1 = time.time()
        print("compute MSE in {} seconds".format(t1-t0))
        return np.mean(error)
    
    def load_matrix(self,rating_tuple):
        t0 = time.time()
        counts = sparse.dok_matrix((self.num_users, self.num_products), dtype=float)
        for i, r_tuple in enumerate(rating_tuple):
            user, item, count = r_tuple
            if user >= self.num_users:
                continue
            if item >= self.num_products:
                continue
            if count != 0:
                counts[user, item] = count
            if i % 1000000 == 0:
                print('loaded %i counts...' % i)
        counts = counts.tocsr()
        t1 = time.time()
        print('Finished loading matrix in %f seconds' % (t1 - t0))
        return counts
    def save_UV(self, dir_path,idx):
        file_u = dir_path + "/matrix_u"+idx+".npz"
        file_v = dir_path + "/matrix_v"+idx+".npz"
        sparse.save_npz(file_u, self.U)
        sparse.save_npz(file_v, self.V)

In [7]:
# lambda_u, lambda_v, K, 
print(num_users,num_items)

142517 111176


In [30]:
resume_point = 0 # how many updates have run
K = 200

log_file = './cf_result/K'+str(K)'.log'
n_iter = 5

save_dir = "./cf_result/cf_result_K"+str(K)
if(resume_point):
    cf = CF(train_counts, num_users, num_items, K=K,checkpoint=save_dir,idx=str(resume_point-1))
else:
    cf = CF(train_counts, num_users, num_items, K=K)
logging.basicConfig(level=logging.INFO,
                format='%(asctime)s - %(message)s',
                datefmt='%m-%d %H:%M',
                filename= log_file)
error = []
for i in range(resume_point,n_iter):
    val_error = cf.MSE(im_tuple)
    error.append(val_error)
    print(time.strftime("%H:%M:%S", time.localtime()) + " iter: {}, validation error: {} ".format(i,val_error))
    logging.info(" iter: {}, validation error: {} ".format(i,val_error))
    cf.fit()
    cf.save_UV(save_dir,str(i))
    print(time.strftime("%H:%M:%S", time.localtime()) +  "save UV to {}".format(save_dir))

generate random UV in 16.850720405578613 seconds
compute MSE in 90.56756782531738 seconds
00:29:23 iter: 0, validation error: 68.97728395234195 
Begin update user
00:29:31: The 0th user 
00:31:16: The 5000th user 
00:32:50: The 10000th user 


KeyboardInterrupt: 

## Predict

In [125]:
# im_test
cf = CF(train_counts,num_users,num_items)
error = []
for user,product,target_rate in im_tuple:
    if(user >= cf.num_users):
        print("new user")
        u_vector = sparse.csr_matrix(cf.U[0].shape, dtype=float)
    else:
        u_vector = cf.U[user]
    if( product>= cf.num_products):
        print("new product")
        v_vector = sparse.csr_matrix(cf.V[0].shape, dtype=float)
    else:
        v_vector = cf.V[product]
    r = cf.predict_rate(user,product)
    print("true rate is {}, predict rate is {}".format(target_rate,r))
    if(len(r)):
        error.append((r-target_rate)**2)

generate random UV in 14.973175287246704 seconds
true rate is 5.0, predict rate is [ 73.8732279]
true rate is 5.0, predict rate is [-86.45384233]
true rate is 5.0, predict rate is [-118.92990619]
true rate is 5.0, predict rate is [ 27.6853184]
true rate is 4.0, predict rate is [ 78.58145105]
true rate is 5.0, predict rate is [ 96.46041114]
true rate is 5.0, predict rate is [ 48.2051795]
true rate is 3.0, predict rate is [-35.86523215]
true rate is 3.0, predict rate is [-108.66322659]
true rate is 5.0, predict rate is [-2.23652494]
true rate is 4.0, predict rate is [-0.60709399]
true rate is 5.0, predict rate is [ 45.62607477]
true rate is 5.0, predict rate is [ 14.22703665]
true rate is 5.0, predict rate is [ 16.73071848]
true rate is 5.0, predict rate is [ 122.65385779]
true rate is 5.0, predict rate is [-73.13365088]
true rate is 3.0, predict rate is [-37.73680144]
true rate is 3.0, predict rate is [ 2.25391273]
true rate is 5.0, predict rate is [ 6.32551209]
true rate is 5.0, predic

true rate is 5.0, predict rate is [-8.98827727]
true rate is 5.0, predict rate is [-122.00175168]
true rate is 5.0, predict rate is [-10.13346776]
true rate is 5.0, predict rate is [ 9.13975846]
true rate is 5.0, predict rate is [ 79.46767386]
true rate is 4.0, predict rate is [ 91.13202849]
true rate is 1.0, predict rate is [ 55.81348293]
true rate is 5.0, predict rate is [-47.17606128]
true rate is 4.0, predict rate is [-39.59519454]
true rate is 5.0, predict rate is [-109.84596118]
true rate is 4.0, predict rate is [ 113.94229073]
true rate is 5.0, predict rate is [ 86.8992575]
true rate is 5.0, predict rate is [ 19.12560953]
true rate is 5.0, predict rate is [-61.69761334]
true rate is 5.0, predict rate is [ 13.21412017]
true rate is 5.0, predict rate is [-21.48950155]
true rate is 5.0, predict rate is [-11.03754983]
true rate is 5.0, predict rate is [ 121.66115635]
true rate is 4.0, predict rate is [ 111.80632211]
true rate is 5.0, predict rate is [ 40.73636428]
true rate is 5.0, 

true rate is 5.0, predict rate is [-5.59930956]
true rate is 5.0, predict rate is [ 10.18779077]
true rate is 5.0, predict rate is [-149.90584699]
true rate is 5.0, predict rate is [-55.91778206]
true rate is 5.0, predict rate is [ 44.1612086]
true rate is 5.0, predict rate is [ 18.00899407]
true rate is 5.0, predict rate is [ 27.56558012]
true rate is 5.0, predict rate is [ 71.78735754]
true rate is 5.0, predict rate is [ 55.40945939]
true rate is 5.0, predict rate is [ 5.67873939]
true rate is 5.0, predict rate is [-6.84238509]
true rate is 5.0, predict rate is [ 61.30086082]
true rate is 5.0, predict rate is [-27.43317455]
true rate is 5.0, predict rate is [ 70.5317988]
true rate is 5.0, predict rate is [-68.66922174]
true rate is 4.0, predict rate is [-37.43706724]
true rate is 1.0, predict rate is [ 94.90532833]
true rate is 3.0, predict rate is [-8.57188509]
true rate is 5.0, predict rate is [-36.12523176]
true rate is 5.0, predict rate is [-57.67826474]
true rate is 5.0, predict

true rate is 5.0, predict rate is [ 31.46643506]
true rate is 5.0, predict rate is [ 22.39093368]
true rate is 3.0, predict rate is [-43.61534865]
true rate is 4.0, predict rate is [ 126.90284669]
true rate is 5.0, predict rate is [-52.06678866]
true rate is 5.0, predict rate is [-35.54289088]
true rate is 5.0, predict rate is [-58.35841192]
true rate is 5.0, predict rate is [-62.87363669]
true rate is 5.0, predict rate is [ 39.13226786]
true rate is 4.0, predict rate is [-25.14834624]
true rate is 5.0, predict rate is [ 41.5082583]
true rate is 5.0, predict rate is [-86.57237386]
true rate is 5.0, predict rate is [ 51.61583708]
true rate is 5.0, predict rate is [ 78.81079584]
true rate is 5.0, predict rate is [ 53.07275936]
true rate is 4.0, predict rate is [ 36.03310246]
true rate is 5.0, predict rate is [-70.1307579]
true rate is 4.0, predict rate is [ 11.70957607]
true rate is 1.0, predict rate is [ 11.14196068]
true rate is 4.0, predict rate is [-94.08548999]
true rate is 3.0, pre

true rate is 5.0, predict rate is [ 73.14534101]
true rate is 4.0, predict rate is [-40.54492054]
true rate is 5.0, predict rate is [-114.25949584]
true rate is 4.0, predict rate is [-83.49881725]
true rate is 1.0, predict rate is [-37.46071805]
true rate is 4.0, predict rate is [ 50.83440713]
true rate is 3.0, predict rate is [ 29.32015361]
true rate is 1.0, predict rate is [-80.48440031]
true rate is 5.0, predict rate is [ 5.45432732]
true rate is 4.0, predict rate is [ 11.25308755]
true rate is 5.0, predict rate is [ 48.22465943]
true rate is 4.0, predict rate is [ 5.26017173]
true rate is 5.0, predict rate is [ 50.89879689]
true rate is 4.0, predict rate is [-46.96220719]
true rate is 5.0, predict rate is [-2.10275133]
true rate is 4.0, predict rate is [-28.66184936]
true rate is 5.0, predict rate is [ 61.05483088]
true rate is 5.0, predict rate is [ 5.37938186]
true rate is 4.0, predict rate is [ 4.13888671]
true rate is 1.0, predict rate is [-37.94310963]
true rate is 4.0, predic

...

true rate is 5.0, predict rate is [-26.77023094]
true rate is 5.0, predict rate is [ 54.2224651]
true rate is 5.0, predict rate is [-16.08786383]
true rate is 5.0, predict rate is [ 82.5052397]
true rate is 4.0, predict rate is [ 9.51507612]
true rate is 5.0, predict rate is [-137.41445071]
true rate is 1.0, predict rate is [ 8.21045322]
true rate is 3.0, predict rate is [ 125.51607021]
true rate is 3.0, predict rate is [ 24.52682706]
true rate is 4.0, predict rate is [-141.03101549]
true rate is 5.0, predict rate is [ 19.32371724]
true rate is 5.0, predict rate is [-16.75582457]
true rate is 4.0, predict rate is [-11.42066754]
true rate is 4.0, predict rate is [ 11.57168489]
true rate is 1.0, predict rate is [-15.02713767]
true rate is 5.0, predict rate is [ 10.37612199]
true rate is 5.0, predict rate is [-73.77511639]
true rate is 4.0, predict rate is [ 102.08976517]
true rate is 2.0, predict rate is [-78.93298431]
true rate is 5.0, predict rate is [-60.95919914]
true rate is 4.0, pr

true rate is 2.0, predict rate is [ 21.3419782]
true rate is 4.0, predict rate is [ 43.31146655]
true rate is 3.0, predict rate is [ 142.98028762]
true rate is 3.0, predict rate is [-6.20664627]
true rate is 5.0, predict rate is [-60.16314568]
true rate is 5.0, predict rate is [ 114.78386352]
true rate is 5.0, predict rate is [-46.31356473]
true rate is 5.0, predict rate is [-48.66871833]
true rate is 5.0, predict rate is [ 49.70179791]
true rate is 3.0, predict rate is [ 38.05588131]
true rate is 3.0, predict rate is [-47.16412411]
true rate is 5.0, predict rate is [ 107.92801087]
true rate is 4.0, predict rate is [ 57.30773071]
true rate is 5.0, predict rate is [ 70.65111011]
true rate is 5.0, predict rate is [ 107.40434233]
true rate is 1.0, predict rate is [ 24.85968698]
true rate is 5.0, predict rate is [-73.25695016]
true rate is 5.0, predict rate is [-87.9421348]
true rate is 4.0, predict rate is [ 19.86721787]
true rate is 5.0, predict rate is [-36.54617121]
true rate is 5.0, p

true rate is 4.0, predict rate is [ 19.83379689]
true rate is 4.0, predict rate is [-140.93735468]
true rate is 4.0, predict rate is [-35.55651895]
true rate is 4.0, predict rate is [ 13.15490425]
true rate is 5.0, predict rate is [-49.59927681]
true rate is 5.0, predict rate is [ 34.51629743]
true rate is 4.0, predict rate is [-49.25418376]
true rate is 5.0, predict rate is [ 13.67920081]
true rate is 3.0, predict rate is [-40.65664409]
true rate is 4.0, predict rate is [-80.8203906]
true rate is 5.0, predict rate is [ 88.62173805]
true rate is 5.0, predict rate is [-29.20842998]
true rate is 5.0, predict rate is [ 43.48379545]
true rate is 5.0, predict rate is [ 20.22015587]
true rate is 2.0, predict rate is [ 30.38382235]
true rate is 4.0, predict rate is [ 64.0783214]
true rate is 4.0, predict rate is [-46.69179277]
true rate is 1.0, predict rate is [ 44.08489953]
true rate is 4.0, predict rate is [ 58.86113299]
true rate is 3.0, predict rate is [-65.62461608]
true rate is 5.0, pre

true rate is 4.0, predict rate is [ 32.3003882]
true rate is 4.0, predict rate is [-45.00678748]
true rate is 4.0, predict rate is [ 54.71972883]
true rate is 3.0, predict rate is [-50.19620084]
true rate is 5.0, predict rate is [ 9.53767823]
true rate is 5.0, predict rate is [-82.06288783]
true rate is 5.0, predict rate is [ 61.52175432]
true rate is 5.0, predict rate is [ 10.69968025]
true rate is 3.0, predict rate is [-54.7185518]
true rate is 5.0, predict rate is [ 10.32392663]
true rate is 4.0, predict rate is [ 33.35866777]
true rate is 4.0, predict rate is [-25.40157073]
true rate is 3.0, predict rate is [-49.48954781]
true rate is 5.0, predict rate is [-7.51555374]
true rate is 5.0, predict rate is [ 36.15749965]
true rate is 3.0, predict rate is [-16.46012404]
true rate is 5.0, predict rate is [ 103.80074167]
true rate is 5.0, predict rate is [ 54.30849605]
true rate is 5.0, predict rate is [-42.7750409]
true rate is 5.0, predict rate is [-42.71895369]
true rate is 4.0, predic

KeyboardInterrupt: 

# To Do 
### 1. save U and V
finished
### 2. integrate small and big data to prince
finished
### 3. if initial U and V randomly, what's the range of the predict confidence would be. so far away with the true value. why necessary to random it, just set as 0?
I have adjust the lambda_u and lambda _v to a more reasonable value. But still need cross validation to tune the hyperparameter

We could not set it 0, otherwise, we couldn't begin first step
### 4.how to make sure trained U V product could be in range(1,5)
adjust regularization parameter, change reference paper, since it's explicit rating, maybe different

### 5. MSE is intractable for large data,solve this or make data small
I solve this by making data small

### 6. speed up
sparse slice principles, change csr to lil matrix. Note to transfer it back.

### 7. set log, store error, store UV for each step.
2. make $U^TV$ close to r(real rating)
3. verify parameters
4. sparse slice principles, lil matrix
5. calculate MAP and  NDCG
6. cross validation to choose hyper parameter: for now, set more reasonable and efficient
7. CTR